In [1]:
from datetime import date
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import yfinance as yf
from ta.momentum import rsi

pd.options.mode.copy_on_write=True
import joblib


def fetch_ticker(ticker="^GSPC", start="1990-01-01"):
    end = date.today()
    p = Path(f"cache/{ticker}.csv")
    if not p.is_file() or date.fromtimestamp(p.stat().st_mtime)<date.today():
        data = yf.download(ticker, start=start, end=end)
        data.columns = data.columns.droplevel(-1)
        data.to_csv(p)
    else:
        data = pd.read_csv(p, index_col=0, parse_dates=[0])
    return data
vix_data = fetch_ticker("^VIX")

In [2]:
df = fetch_ticker("^GSPC")
df_weekly = df.resample('W').agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'})
df_monthly = df.resample('M').agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'})

/tmp/ipykernel_75751/2957332899.py:3: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df_monthly = df.resample('M').agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'})


In [4]:
from scipy.stats import kurtosis, skew

print(f"Skewness: {skew(df['Close'])}, Kurtosis: {kurtosis(df['Close'])}")


Skewness: 1.3968165248546731, Kurtosis: 1.3169608273482138


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from ta.trend import MACD

df["MA50"] = df["Close"].rolling(window=50).mean()
df["MA200"] = df["Close"].rolling(window=200).mean()
df["20MA"] = df["Close"].rolling(window=20).mean()
df["20STD"] = df["Close"].rolling(window=20).std()
df["UpperBand"] = df["20MA"] + (2 * df["20STD"])
df["LowerBand"] = df["20MA"] - (2 * df["20STD"])
df["RSI"] = rsi(df["Close"], window=14)
macd = MACD(df["Close"], window_fast=12, window_slow=26, window_sign=9)
df["MACD"] = macd.macd()
df["MACD_signal"] = macd.macd_signal()
df["MACD_hist"] = macd.macd_diff()
df["Slope"] = df["Close"].diff()
df["Turning_Points"] = (
    (df["Slope"].shift(1) > 0) & (df["Slope"] < 0)
).astype(int)
df["Acceleration"] = df["Slope"].diff()
df["Momentum"] = df["Close"] - df["Close"].shift(5)
df["DayOfWeek"] = df.index.day_of_week
df["Change"] = df["Close"].pct_change()
df["UpperBreak"] = (df["UpperBand"] < df["Close"]).astype(int)
df["LowerBreak"] = (df["LowerBand"] > df["Close"]).astype(int)
df["MA50_rel"] = df["Close"] - df["MA50"]
df["MA200_rel"] = df["Close"] - df["MA200"]
df["DayOfWeek_sin"] = np.sin(2 * np.pi * df["DayOfWeek"] / 7)
df["DayOfWeek_cos"] = np.cos(2 * np.pi * df["DayOfWeek"] / 7)
df.drop(columns=["DayOfWeek"], inplace=True)

from scipy.signal import find_peaks

# Find peaks (local maxima)
peaks, _ = find_peaks(
    df["Close"], distance=30
)  # Distance=10 ensures minimum separation
troughs, _ = find_peaks(
    -df["Close"], distance=30
)  # Invert to find local minima

peak_indices = df.index[peaks]  # Convert position to index
trough_indices = df.index[troughs]
df["Target"] = 0  # Default: No peak or trough
df.loc[peak_indices, "Target"] = 1  # Mark peaks
df.loc[trough_indices, "Target"] = -1  # Mark troughs
plt.figure(figsize=(40, 24), dpi=400)
plt.plot(df.index, df["Close"], label="Close Price", alpha=0.6)
plt.scatter(
    df.index[peaks],
    df["Close"][peaks],
    color="red",
    label="Peaks",
    marker="^",
    s=100,
)
plt.scatter(
    df.index[troughs],
    df["Close"][troughs],
    color="blue",
    label="Troughs",
    marker="v",
    s=100,
)
plt.legend()
plt.title("Detected Peaks and Troughs")
plt.show()

from hmmlearn.hmm import GaussianHMM

cols_to_use = [
    "Change",  # daily % change of Close
    "Volume",
    "Momentum",
    "MA50_rel",  # moving average 50 relative to Close
    "MA200_rel",  # moving average 200 relative to Close
    "UpperBreak",  # binary flag (0 or 1)
    "LowerBreak",  # binary flag (0 or 1)
    "DayOfWeek_sin",  # binary flag (0 or 1)
    "DayOfWeek_cos",  # binary flag (0 or 1)
]
df_hmm = df[cols_to_use].dropna().copy()

continuous_cols = ["Change", "Volume", "Momentum", "MA50_rel", "MA200_rel"]
binary_cols = ["UpperBreak", "LowerBreak"]
cyclic_cols = ["DayOfWeek_sin", "DayOfWeek_cos"]
scaler = StandardScaler()
df_hmm[continuous_cols] = scaler.fit_transform(df_hmm[continuous_cols])
final_feature_cols = continuous_cols + cyclic_cols + binary_cols
X = df_hmm[final_feature_cols].to_numpy()

print(df_hmm.describe())
n_components = 5  # try 5 first
hmm = GaussianHMM(
    n_components=3, covariance_type="full", n_iter=100000, random_state=42
)
hmm.fit(X)
states = hmm.predict(X)
df.loc[df_hmm.index, "Market_State"] = states
joblib.dump(hmm, "hmm.pkl")
print("HMM training completed. Unique states found:", np.unique(states))

/tmp/ipykernel_75751/4083887818.py:52: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df["Close"][peaks],
/tmp/ipykernel_75751/4083887818.py:60: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df["Close"][troughs],


             Change        Volume      Momentum      MA50_rel     MA200_rel  \
count  7.647000e+03  7.647000e+03  7.647000e+03  7.647000e+03  7.647000e+03   
mean   1.022096e-17 -5.946742e-17 -1.115014e-17 -5.946742e-17  2.973371e-17   
std    1.000065e+00  1.000065e+00  1.000065e+00  1.000065e+00  1.000065e+00   
min   -1.016794e+01 -1.650189e+00 -1.068859e+01 -1.032758e+01 -5.047086e+00   
25%   -4.244366e-01 -9.231492e-01 -3.480381e-01 -3.718404e-01 -4.028036e-01   
50%    2.295235e-02  1.858768e-01  2.630004e-02  2.264999e-02 -2.563790e-02   
75%    4.716681e-01  6.567127e-01  3.900767e-01  4.115550e-01  3.331289e-01   
max    9.757503e+00  4.977044e+00  7.535644e+00  4.206831e+00  3.867852e+00   

        UpperBreak   LowerBreak  DayOfWeek_sin  DayOfWeek_cos  
count  7647.000000  7647.000000    7647.000000    7647.000000  
mean      0.042893     0.046816       0.360846      -0.092004  
std       0.202628     0.211258       0.515667       0.771703  
min       0.000000     0.000000 

In [75]:
df

,Close,High,Low,Open,Volume,Market_State,MA50,MA200,20MA,20STD,...,Acceleration,Momentum,Change,UpperBreak,LowerBreak,MA50_rel,MA200_rel,Target,DayOfWeek_sin,DayOfWeek_cos
Date,,,,,,,,,,,,,,,,,,,,,
1993-12-07,466.760010,466.769989,465.440002,466.429993,285690000,1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,NaN,NaN,0,0.781831,0.623490
1993-12-08,466.290009,466.730011,465.420013,465.880005,314460000,1,NaN,NaN,NaN,NaN,...,NaN,NaN,-0.001007,0,0,NaN,NaN,0,0.974928,-0.222521
1993-12-09,464.179993,466.540009,463.869995,466.290009,287570000,1,NaN,NaN,NaN,NaN,...,-1.640015,NaN,-0.004525,0,0,NaN,NaN,0,0.433884,-0.900969
1993-12-10,463.929993,464.869995,462.660004,464.179993,245620000,1,NaN,NaN,NaN,NaN,...,1.860016,NaN,-0.000539,0,0,NaN,NaN,0,-0.433884,-0.900969
1993-12-13,465.700012,465.709991,462.709991,463.929993,256580000,1,NaN,NaN,NaN,NaN,...,2.020020,NaN,0.003815,0,0,NaN,NaN,0,0.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-31,6040.529785,6120.910156,6030.930176,6096.790039,4843770000,0,5990.105400,5629.953550,5979.515527,92.576349,...,-62.500000,-60.710449,-0.005047,0,0,50.424385,410.576235,0,-0.433884,-0.900969
2025-02-03,5994.569824,6022.129883,5923.930176,5969.649902,4797210000,0,5992.124395,5634.669348,5985.816528,88.839422,...,-15.319824,-17.709961,-0.007609,0,0,2.445430,359.900476,0,0.000000,1.000000
2025-02-04,6037.879883,6042.479980,5990.870117,5998.140137,4410160000,0,5994.542393,5639.747698,5990.587012,88.950890,...,89.270020,-29.820312,0.007225,0,0,43.337490,398.132185,0,0.781831,0.623490


In [59]:
df_vis = df[df.index>'2020-01-01']
# Compute daily % change
df_vis["Daily_Return"] = df_vis["Close"].pct_change() * 100  # Convert to percentage

# Extract Month Names
df_vis["Month"] = df_vis.index.month
df_vis["DayOfWeek"] = df_vis.index.day_of_week

# Compute average monthly % change across all years
monthly_avg_change = df_vis.groupby("Month")["Daily_Return"].median()
daily_avg_change = df_vis.groupby("DayOfWeek")["Daily_Return"].median()

# Rename index for clarity
monthly_avg_change.index = [
    "Jan", "Feb", "Mar", "Apr", "May", "Jun", 
    "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"
]
daily_avg_change.index = [
    "Mon", "Tue", "Wed", "Thu", "Fri"
]

print(monthly_avg_change)
print(daily_avg_change)


Jan    0.058338
Feb    0.072590
Mar    0.101540
Apr   -0.008426
May    0.070346
Jun    0.132816
Jul    0.238220
Aug    0.127574
Sep   -0.131857
Oct    0.013492
Nov    0.229385
Dec    0.017185
Name: Daily_Return, dtype: float64
Mon    0.180008
Tue   -0.009825
Wed    0.088121
Thu    0.090732
Fri    0.081998
Name: Daily_Return, dtype: float64


In [79]:
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import (
    LSTM,
    Add,
    BatchNormalization,
    Conv1D,
    Dense,
    Dropout,
    Input,
    MultiHeadAttention,
    RepeatVector,
    Reshape,
    TimeDistributed,
)
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam

# Prepare Data
lookback = 30  # Predict using past 30 days
scaler = MinMaxScaler()


def create_sequences(data: np.ndarray, lookback: int, forecast_horizon: int, label_col_index: int):
    X, y_reg, y_clf = [], [], []
    num_classes = 3  # assuming 3 classes for classification
    for i in range(len(data) - lookback - forecast_horizon):
        X_seq = data[i: i + lookback]
        future_seq = data[i + lookback: i + lookback + forecast_horizon]
        
        # Regression target: assuming the target column is at label_col_index
        reg_target = future_seq[:, label_col_index]
        
        # Classification target: take the same column and one-hot encode it.
        clf_target = future_seq[:, label_col_index]
        one_hot = np.zeros((forecast_horizon, num_classes))
        for j, label in enumerate(clf_target):
            # Assuming labels are -1, 0, 1; map to indices 0, 1, 2:
            class_index = int(float(label) + 1)
            one_hot[j, class_index] = 1
        
        X.append(X_seq)
        y_reg.append(reg_target)
        y_clf.append(one_hot)
    return np.array(X, dtype=np.float32), np.array(y_reg, dtype=np.float32), np.array(y_clf, dtype=np.float32)



# Suppose these are the feature columns you want to scale:
feature_cols = ["Change", "Volume","Slope", "Acceleration", "MACD", "RSI", "MA200_rel", "MA50_rel"]
# And these two columns are either labels or binary flags:
non_scaled_cols = ["Turning_Points", "UpperBreak", "LowerBreak", "DayOfWeek_sin", "DayOfWeek_cos", "Market_State"]
scaler = StandardScaler()
df_scaled_features = pd.DataFrame(scaler.fit_transform(df[feature_cols]), columns=feature_cols, index=df.index)
df_processed = pd.concat([df_scaled_features, df[non_scaled_cols+["Target"]]], axis=1).dropna()
data_array = df_processed[feature_cols + non_scaled_cols+["Target"]].to_numpy()

lookback = 30  # past 30 days as input
forecast_horizon = 5  # predict next 5 days
X, y_reg, y_clf = create_sequences(
    data_array, lookback, forecast_horizon, label_col_index=len(df_processed.columns)-1
)

def build_deeper_multitask_model(input_shape: tuple[int, int],
                                 forecast_horizon: int,
                                 num_classes: int = 3) -> Model:
    """
    A deeper multi-task LSTM model with additional feature extraction layers.
    """

    inp = Input(shape=input_shape)

    # **1. Convolutional Feature Extractor**
    x = Conv1D(filters=64, kernel_size=3, activation="relu", padding="same")(inp)
    x = BatchNormalization()(x)
    
    # **2. LSTM Layers (Deeper)**
    x = LSTM(256, return_sequences=True)(x)
    x = Dropout(0.2)(x)
    x = LSTM(128, return_sequences=True)(x)
    x = Dropout(0.2)(x)
    x = LSTM(64, return_sequences=False)(x)
    x = Dropout(0.2)(x)

    # **3. Self-Attention (Optional)**
    attention = MultiHeadAttention(num_heads=4, key_dim=64)(x[:, None, :], x[:, None, :])
    x = Add()([x, attention[:, 0, :]])  # Residual connection

    # **4. Regression Head: Predict the next n price changes**
    regression_output = Dense(forecast_horizon, name="regression_output")(x)

    # **5. Classification Head: Predict the movement category**
    repeated = RepeatVector(forecast_horizon)(x)
    x_clf = LSTM(128, return_sequences=True)(repeated)
    classification_output = TimeDistributed(
        Dense(num_classes, activation="softmax"), name="classification_output"
    )(x_clf)

    # **Compile Model**
    model = Model(inputs=inp, outputs=[regression_output, classification_output])
    model.compile(
        optimizer=Adam(learning_rate=4e-6),
        loss={"regression_output": "mse", "classification_output": "categorical_crossentropy"},
        metrics={"regression_output": "mse", "classification_output": "accuracy"},
    )

    return model

def build_multitask_model(
    input_shape: tuple[int, int], forecast_horizon: int, num_classes: int = 3
) -> Model:
    """
    Builds a multi-task LSTM model to predict the next n Price Changes (regression)
    and the corresponding categorical movement (e.g., negative, neutral, positive)
    for each forecast day (classification).

    Args:
        input_shape (tuple[int, int]): Shape of the input sequence (lookback, num_features).
        forecast_horizon (int): Number of future days to predict.
        num_classes (int): Number of classes for classification (default is 3).

    Returns:
        Model: A compiled Keras model with two outputs:
            - "price_change_output": Regression predictions of price changes.
            - "classification_output": Classification predictions (one-hot) for movement.
    """
    inp = Input(shape=input_shape)  # (lookback, num_features)

    # Shared LSTM layers
    x = LSTM(128, return_sequences=True)(inp)
    x = Dropout(0.2)(x)
    x = LSTM(64, return_sequences=False)(x)
    x = Dropout(0.2)(x)

    # Regression head: predict closing prices for forecast_horizon days
    price_change_output = Dense(forecast_horizon, name="price_change_output")(x)

    # Classification head:
    # Repeat the shared representation for each forecast day.
    repeated = RepeatVector(forecast_horizon)(x)
    x_clf = LSTM(64, return_sequences=True)(repeated)
    classification_output = TimeDistributed(
        Dense(num_classes, activation="softmax"),
        name="classification_output"
    )(x_clf)
    
    model = Model(inputs=inp, outputs=[price_change_output, classification_output])
    model.compile(
        optimizer=Adam(learning_rate=2e-5),
        loss={
            "price_change_output": "mse",  # For predicting continuous price changes
            "classification_output": "categorical_crossentropy"  # For categorical movement
        },
        metrics={
            "price_change_output": "mse",
            "classification_output": "accuracy"
        }
    )
    return model


# Define the input shape based on your data
lookback = X.shape[1]  # e.g., 30 days
num_features = X.shape[2]  # e.g., 3 features: Close, Slope, Acceleration
input_shape = (lookback, num_features)
forecast_horizon = y_reg.shape[1]  # e.g., 5 days

model = build_deeper_multitask_model(input_shape, forecast_horizon)

# View the model summary to verify the architecture.
model.summary()

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Callbacks
early_stop = EarlyStopping(
    monitor="val_loss", patience=5, restore_best_weights=True
)
checkpoint = ModelCheckpoint(
    "multitask_model.keras", monitor="val_loss", save_best_only=True
)

history = model.fit(
    X,
    {"regression_output": y_reg, "classification_output": y_clf},
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stop, checkpoint],
)

joblib.dump(scaler, "scaler.pkl")

Model: "functional_15"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_15      │ (None, 30, 15)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_2 (Conv1D)   │ (None, 30, 64)    │      2,944 │ input_layer_15[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 30, 64)    │        256 │ conv1d_2[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_47 (LSTM)      │ (None, 30, 256)   │    328,704 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_34          │ (None, 30, 256)   │          0 │ lstm_47[0][0]     │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_48 (LSTM)      │ (None, 30, 128)   │    197,120 │ dropout_34[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_35          │ (None, 30, 128)   │          0 │ lstm_48[0][0]     │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_49 (LSTM)      │ (None, 64)        │     49,408 │ dropout_35[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_36          │ (None, 64)        │          0 │ lstm_49[0][0]     │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_6          │ (None, 1, 64)     │          0 │ dropout_36[0][0]  │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_7          │ (None, 1, 64)     │          0 │ dropout_36[0][0]  │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 1, 64)     │     66,368 │ get_item_6[0][0], │
│ (MultiHeadAttentio… │                   │            │ get_item_7[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_8          │ (None, 64)        │          0 │ multi_head_atten… │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 64)        │          0 │ dropout_36[0][0], │
│                     │                   │            │ get_item_8[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ repeat_vector_15    │ (None, 5, 64)     │          0 │ add_2[0][0]       │
│ (RepeatVector)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_50 (LSTM)      │ (None, 5, 128)    │     98,816 │ repeat_vector_15… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ regression_output   │ (None, 5)         │        325 │ add_2[0][0]       │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ classification_out… │ (None, 5, 3)      │        387 │ lstm_50[0][0]     │
│ (TimeDistributed)   │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴─────────────────

 Total params: 744,328 (2.84 MB)

 Trainable params: 744,200 (2.84 MB)

 Non-trainable params: 128 (512.00 B)

Epoch 1/50
191/191 ━━━━━━━━━━━━━━━━━━━━ 11s 44ms/step - classification_output_accuracy: 0.5412 - classification_output_loss: 1.0834 - loss: 1.1475 - regression_output_loss: 0.0640 - regression_output_mse: 0.0640 - val_classification_output_accuracy: 0.7100 - val_classification_output_loss: 1.0703 - val_loss: 1.1345 - val_regression_output_loss: 0.0646 - val_regression_output_mse: 0.0641
Epoch 2/50
191/191 ━━━━━━━━━━━━━━━━━━━━ 9s 46ms/step - classification_output_accuracy: 0.8323 - classification_output_loss: 1.0089 - loss: 1.0702 - regression_output_loss: 0.0613 - regression_output_mse: 0.0613 - val_classification_output_accuracy: 0.8071 - val_classification_output_loss: 1.0222 - val_loss: 1.0934 - val_regression_output_loss: 0.0719 - val_regression_output_mse: 0.0714
Epoch 3/50
191/191 ━━━━━━━━━━━━━━━━━━━━ 8s 41ms/step - classification_output_accuracy: 0.8972 - classification_output_loss: 0.7990 - loss: 0.8652 - regression_output_loss: 0.0661 - regression_output_mse: 0.0661 - val_clas

['scaler.pkl']

In [35]:
# Load the saved model when needed.
model = tf.keras.models.load_model("multitask_model.keras")

# Suppose new_sequence is of shape (1, lookback, num_features)
# For example, using the latest data:
new_sequence = X[-1:].copy()  # this is just an example

predicted_regression, predicted_classification = model.predict(new_sequence)

# predicted_regression: predicted closing prices for next forecast_horizon days.
# predicted_classification: predicted probabilities for each class (per day).
# You might choose the class with the highest probability:
predicted_classes = predicted_classification.argmax(axis=-1)

print("Predicted Closing Prices:", predicted_regression)
print("Predicted Classes (0=trough, 1=neutral, 2=peak):", predicted_classes)


ValueError: Invalid dtype: object

In [110]:
X

array([[[ 3.03230011e+02,  3.20001221e+00, -1.36999512e+00,
          0.00000000e+00],
        [ 2.98920013e+02, -4.30999756e+00, -7.51000977e+00,
          0.00000000e+00],
        [ 2.98760010e+02, -1.60003662e-01,  4.14999390e+00,
          0.00000000e+00],
        ...,
        [ 3.16029999e+02,  7.20001221e-01,  4.75000000e+00,
          0.00000000e+00],
        [ 3.15100006e+02, -9.29992676e-01, -1.64999390e+00,
         -1.00000000e+00],
        [ 3.16510010e+02,  1.41000366e+00,  2.33999634e+00,
          0.00000000e+00]],

       [[ 2.98920013e+02, -4.30999756e+00, -7.51000977e+00,
          0.00000000e+00],
        [ 2.98760010e+02, -1.60003662e-01,  4.14999390e+00,
          0.00000000e+00],
        [ 3.05739990e+02,  6.97998047e+00,  7.13998413e+00,
          0.00000000e+00],
        ...,
        [ 3.15100006e+02, -9.29992676e-01, -1.64999390e+00,
         -1.00000000e+00],
        [ 3.16510010e+02,  1.41000366e+00,  2.33999634e+00,
          0.00000000e+00],
        [ 3.181

In [61]:
model.classes_

array([-1,  0,  1])